<a href="https://colab.research.google.com/github/Hashawaji/portrait-video-synthesis/blob/main/clustering/3_clustering_hierarchical/Hierarchical_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import zipfile
from tqdm import tqdm
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F

from collections import defaultdict
import pickle
from sklearn.cluster import KMeans
import random

In [ ]:
clusters_info = 'Sawaiz_2/pkl_for_lstm_encoded/17_53_50_800'
file_path = '/content/drive/MyDrive/'+clusters_info+'/live_portrait_descriptors_all_encoder.pkl'

# Open the file in binary read mode and load the data
with open(file_path, 'rb') as file:
    frames_input_data = pickle.load(file)

In [ ]:

def cluster_recursively(vectors, frames, num_clusters):

    kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)
    labels = kmeans.labels_

    # Group frames by their assigned cluster labels
    cluster_indices = {i: [] for i in range(num_clusters)}
    cluster_vectors = {i: [] for i in range(num_clusters)}

    for frame, label, vector in zip(frames, labels, vectors):
        cluster_indices[label].append(frame)
        cluster_vectors[label].append(vector)

    # Calculate representatives (centroids) for each cluster
    cluster_representatives = {}
    for cluster_id, vectors in cluster_vectors.items():
        cluster_vectors_np = np.array(vectors)  # Convert to numpy array
        if cluster_vectors_np.size > 0:        # Avoid empty clusters
            centroid = cluster_vectors_np.mean(axis=0)
            cluster_representatives[cluster_id] = centroid
        else:
            cluster_representatives[cluster_id] = None

    for cluster_id, frame_list in cluster_indices.items():
        print(f"Cluster {cluster_id}: {len(frame_list)} frames")
    return cluster_indices, cluster_vectors, cluster_representatives

def multi_level_kmeans(frame_to_vector, num_clusters=4):

    # Extract keys (frame file paths) and vectors (embeddings)
    frames = list(frame_to_vector.keys())
    vectors = np.array([frame_to_vector[frame] for frame in frames])

    # Function to recursively perform K-means clustering

    return cluster_recursively(vectors,frames, num_clusters)

csv_data = {}
csv_data['Level 1'] = {"Names":[], "Sizes":[]}
csv_data['Level 2'] = {"Names":[], "Sizes":[]}
csv_data['Level 3'] = {"Names":[], "Sizes":[]}
csv_data['Level 4'] = {"Names":[], "Sizes":[]}

## First level
num_clusters=4

print(f"Total Clusters: {num_clusters}")

# Only first level clustering
cluster_indices, cluster_vectors, cs = multi_level_kmeans(frames_input_data, num_clusters)

clusters_rep = {}
frame_to_cluster = {}


names = []
sizes = []
for key, val in cluster_indices.items():
    names.append(f"Cluster_{key}")
    sizes.append(len(val))
    clusters_rep[f"Cluster_{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{key}"

csv_data['Level 1']["Names"] = names
csv_data['Level 1']["Sizes"] = sizes


Total Clusters: 4
Cluster 0: 151578 frames
Cluster 1: 418247 frames
Cluster 2: 463100 frames
Cluster 3: 123218 frames


In [ ]:
def dividing2ndlevel(cluster_data):
  result = {}
  for key, val in cluster_data.items():
    length = len(val)
    if length >= 25000 and length <= 40000:
      result[key] = 2
    elif length > 40000 and length <= 60000:
      result[key] = 3
    elif length > 60000 and length <= 80000:
      result[key] = 4
    elif length > 80000:
      result[key] = 5
    else:
      result[key] = 1
  return result


def dividing3rdlevel(cluster_data):
  result = {}
  for key, val in cluster_data.items():
    length = len(val)
    if length >= 5000 and length <= 10000:
      result[key] = 2
    elif length > 10000 and length <= 15000:
      result[key] = 3
    elif length > 15000 and length <= 20000:
      result[key] = 4
    elif length > 20000:
      result[key] = 5
    else:
      result[key] = 1
  return result


In [ ]:
# This is for creating the second level for 1 node of first level.
# You have to do for each node below
cluster_id = 0
print(f"Cluster {cluster_id}")
a,b,cs = cluster_recursively(cluster_vectors[cluster_id], cluster_indices[cluster_id], 4) # 4 means it will make 4 childs

level_2_names = []
level_2_sizes =[]
for key, val in a.items():
    level_2_names.append(f"Cluster_{cluster_id}.{key}")
    level_2_sizes.append(len(val))
    clusters_rep[f"Cluster_{cluster_id}.{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{cluster_id}.{key}"
csv_data['Level 2']["Names"].extend(level_2_names)
csv_data['Level 2']["Sizes"].extend(level_2_sizes)

# subdivision1 = dividing2ndlevel(a)
# for key1, val1 in subdivision1.items():
#   print(f"Sub-Cluster {key1}")
#   a1, b1,_ = cluster_recursively( b[key1], a[key1], val1)
#   for key, val in a1.items():
#     csv_data['Level 3']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key}")
#     csv_data['Level 3']["Sizes"].append(len(val))
#     # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key}"] = cs[key]
#     # for i in val:
#     #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key}"
#   subdivision2 = dividing3rdlevel(a1)
#   for key2, val2 in subdivision2.items():
#     print(f"Sub-Sub Cluster {key2}")
#     a2, b2, _ = cluster_recursively( b1[key2], a1[key2], val2)
#     for key, val in a2.items():
#       csv_data['Level 4']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key2}.{key}")
#       csv_data['Level 4']["Sizes"].append(len(val))
#       # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key2}.{key}"] = cs[key]
#       # for i in val:
#       #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key2}.{key}"

Cluster 0
Cluster 0: 21462 frames
Cluster 1: 48954 frames
Cluster 2: 64120 frames
Cluster 3: 17042 frames


In [ ]:
cluster_id = 1
print(f"Cluster {cluster_id}")
a,b,cs = cluster_recursively( cluster_vectors[cluster_id], cluster_indices[cluster_id], 8)
level_2_names = []
level_2_sizes =[]
for key, val in a.items():
    level_2_names.append(f"Cluster_{cluster_id}.{key}")
    level_2_sizes.append(len(val))
    clusters_rep[f"Cluster_{cluster_id}.{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{cluster_id}.{key}"
csv_data['Level 2']["Names"].extend(level_2_names)
csv_data['Level 2']["Sizes"].extend(level_2_sizes)

# subdivision1 = dividing2ndlevel(a)
# for key1, val1 in subdivision1.items():
#   print(f"Sub-Cluster {key1}")
#   a1, b1,cs = cluster_recursively( b[key1], a[key1], val1)
#   for key, val in a1.items():
#     csv_data['Level 3']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key}")
#     csv_data['Level 3']["Sizes"].append(len(val))
#     # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key}"] = cs[key]
#     # for i in val:
#     #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key}"
#   subdivision2 = dividing3rdlevel(a1)
#   for key2, val2 in subdivision2.items():
#     print(f"Sub-Sub Cluster {key2}")
#     a2, b2, _ = cluster_recursively( b1[key2], a1[key2], val2)
#     for key, val in a2.items():
#       csv_data['Level 4']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key2}.{key}")
#       csv_data['Level 4']["Sizes"].append(len(val))
#       # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key2}.{key}"] = cs[key]
#       # for i in val:
#       #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key2}.{key}"

Cluster 1
Cluster 0: 2478 frames
Cluster 1: 58085 frames
Cluster 2: 54110 frames
Cluster 3: 73256 frames
Cluster 4: 82115 frames
Cluster 5: 39441 frames
Cluster 6: 50466 frames
Cluster 7: 58296 frames


In [ ]:
cluster_id = 2
print(f"Cluster {cluster_id}")
a,b, cs = cluster_recursively( cluster_vectors[cluster_id], cluster_indices[cluster_id], 8)
level_2_names = []
level_2_sizes =[]
for key, val in a.items():
    level_2_names.append(f"Cluster_{cluster_id}.{key}")
    level_2_sizes.append(len(val))
    clusters_rep[f"Cluster_{cluster_id}.{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{cluster_id}.{key}"
csv_data['Level 2']["Names"].extend(level_2_names)
csv_data['Level 2']["Sizes"].extend(level_2_sizes)

# subdivision1 = dividing2ndlevel(a)
# for key1, val1 in subdivision1.items():
#   print(f"Sub-Cluster {key1}")
#   a1, b1,  cs = cluster_recursively( b[key1], a[key1], val1)
#   for key, val in a1.items():
#     csv_data['Level 3']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key}")
#     csv_data['Level 3']["Sizes"].append(len(val))
#     # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key}"] = cs[key]
#     # for i in val:
#     #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key}"
#   subdivision2 = dividing3rdlevel(a1)
#   for key2, val2 in subdivision2.items():
#     print(f"Sub-Sub Cluster {key2}")
#     a2, b2, _ = cluster_recursively( b1[key2], a1[key2], val2)
#     for key, val in a2.items():
#       csv_data['Level 4']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key2}.{key}")
#       csv_data['Level 4']["Sizes"].append(len(val))
#       # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key2}.{key}"] = cs[key]
#       # for i in val:
#       #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key2}.{key}"

Cluster 2
Cluster 0: 80230 frames
Cluster 1: 61487 frames
Cluster 2: 23091 frames
Cluster 3: 59732 frames
Cluster 4: 100530 frames
Cluster 5: 27730 frames
Cluster 6: 51469 frames
Cluster 7: 58831 frames


In [ ]:
cluster_id = 3
print(f"Cluster {cluster_id}")
a,b,cs = cluster_recursively( cluster_vectors[cluster_id], cluster_indices[cluster_id], 4)
level_2_names = []
level_2_sizes =[]
for key, val in a.items():
    level_2_names.append(f"Cluster_{cluster_id}.{key}")
    level_2_sizes.append(len(val))
    clusters_rep[f"Cluster_{cluster_id}.{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{cluster_id}.{key}"
csv_data['Level 2']["Names"].extend(level_2_names)
csv_data['Level 2']["Sizes"].extend(level_2_sizes)

# subdivision1 = dividing2ndlevel(a)
# for key1, val1 in subdivision1.items():
#   print(f"Sub-Cluster {key1}")
#   a1, b1,cs = cluster_recursively( b[key1], a[key1], val1)
#   for key, val in a1.items():
#     csv_data['Level 3']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key}")
#     csv_data['Level 3']["Sizes"].append(len(val))
#     # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key}"] = cs[key]
#     # for i in val:
#     #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key}"
#   subdivision2 = dividing3rdlevel(a1)
#   for key2, val2 in subdivision2.items():
#     print(f"Sub-Sub Cluster {key2}")
#     a2, b2, _ = cluster_recursively( b1[key2], a1[key2], val2)
#     for key, val in a2.items():
#       csv_data['Level 4']["Names"].append(f"Cluster_{cluster_id}.{key1}.{key2}.{key}")
#       csv_data['Level 4']["Sizes"].append(len(val))
#       # clusters_rep[f"Cluster_{cluster_id}.{key1}.{key2}.{key}"] = cs[key]
#       # for i in val:
#       #   frame_to_cluster[i] = f"Cluster_{cluster_id}.{key1}.{key2}.{key}"


Cluster 3
Cluster 0: 19785 frames
Cluster 1: 34900 frames
Cluster 2: 56041 frames
Cluster 3: 12492 frames


In [ ]:
cluster_id = 4
print(f"Cluster {cluster_id}")
a,b,cs = cluster_recursively( cluster_vectors[cluster_id], cluster_indices[cluster_id], 4)
level_2_names = []
level_2_sizes =[]
for key, val in a.items():
    level_2_names.append(f"Cluster_{cluster_id}.{key}")
    level_2_sizes.append(len(val))
    clusters_rep[f"Cluster_{cluster_id}.{key}"] = cs[key]
    for i in val:
      frame_to_cluster[i] = f"Cluster_{cluster_id}.{key}"
csv_data['Level 2']["Names"].extend(level_2_names)
csv_data['Level 2']["Sizes"].extend(level_2_sizes)

Cluster 4
Cluster 0: 35330 frames
Cluster 1: 44798 frames
Cluster 2: 9054 frames
Cluster 3: 43981 frames


In [ ]:
print(clusters_rep.keys())

dict_keys(['Cluster_0', 'Cluster_1', 'Cluster_2', 'Cluster_3', 'Cluster_4', 'Cluster_0.0', 'Cluster_0.1', 'Cluster_0.2', 'Cluster_1.0', 'Cluster_1.1', 'Cluster_1.2', 'Cluster_1.3', 'Cluster_1.4', 'Cluster_1.5', 'Cluster_1.6', 'Cluster_2.0', 'Cluster_2.1', 'Cluster_2.2', 'Cluster_2.3', 'Cluster_2.4', 'Cluster_2.5', 'Cluster_2.6', 'Cluster_3.0', 'Cluster_3.1', 'Cluster_3.2', 'Cluster_4.0', 'Cluster_4.1', 'Cluster_4.2', 'Cluster_4.3'])


In [ ]:
with open("/content/drive/MyDrive/Sawaiz_2/pkl_for_lstm_encoded/17_53_50_800/frame_to_cluster_clusters4_level1_clusters24_level2.pkl", "wb") as file:
    pickle.dump(frame_to_cluster, file)

In [ ]:
with open("/content/drive/MyDrive/Sawaiz_2/pkl_for_lstm_encoded/17_53_50_800/cluster_rep_clusters4_level1_clusters24_level2.pkl", "wb") as file:
    pickle.dump(clusters_rep, file)

In [ ]:
def create_csv_from_levels(csv_data, output_filename="output.csv"):
    # Determine the maximum number of rows needed (longest level)
    max_rows = max(len(data["Names"]) for data in csv_data.values())

    # Initialize an empty dictionary for creating the DataFrame
    columns = {}

    for level, data in csv_data.items():
        # Add columns for Names and Sizes for the current level
        columns[f"{level} Names"] = data["Names"] + [""] * (max_rows - len(data["Names"]))
        columns[f"{level} Sizes"] = data["Sizes"] + [""] * (max_rows - len(data["Sizes"]))

    # Create a DataFrame
    df = pd.DataFrame(columns)

    # Save to a CSV file
    df.to_csv(output_filename, index=False)
    print(f"CSV saved to {output_filename}")

# Call the function
create_csv_from_levels(csv_data)

CSV saved to output.csv


In [ ]:
'''
Total Clusters: 3
Cluster 0: 376935 frames
Cluster 1: 637993 frames
Cluster 2: 141215 frames



Total Clusters: 4
Cluster 0: 151578 frames
Cluster 1: 418247 frames
Cluster 2: 463100 frames
Cluster 3: 123218 frames



Total Clusters: 5
Cluster 0: 110479 frames
Cluster 1: 352769 frames
Cluster 2: 442863 frames
Cluster 3: 116869 frames
Cluster 4: 133163 frames



Total Clusters: 6
Cluster 0: 103997 frames
Cluster 1: 305352 frames
Cluster 2: 371010 frames
Cluster 3: 71429 frames
Cluster 4: 133202 frames
Cluster 5: 171153 frames



Total Clusters: 7
Cluster 0: 74753 frames
Cluster 1: 249606 frames
Cluster 2: 287714 frames
Cluster 3: 66731 frames
Cluster 4: 116677 frames
Cluster 5: 159373 frames
Cluster 6: 201289 frames



Total Clusters: 8
Cluster 0: 235871 frames
Cluster 1: 160281 frames
Cluster 2: 211026 frames
Cluster 3: 55867 frames
Cluster 4: 277021 frames
Cluster 5: 113273 frames
Cluster 6: 79208 frames
Cluster 7: 23596 frames



Total Clusters: 9
Cluster 0: 194435 frames
Cluster 1: 130506 frames
Cluster 2: 224215 frames
Cluster 3: 60201 frames
Cluster 4: 244565 frames
Cluster 5: 107907 frames
Cluster 6: 72159 frames
Cluster 7: 17146 frames
Cluster 8: 105009 frames



Total Clusters: 10
Cluster 0: 116710 frames
Cluster 1: 201119 frames
Cluster 2: 168595 frames
Cluster 3: 59084 frames
Cluster 4: 249743 frames
Cluster 5: 89068 frames
Cluster 6: 27390 frames
Cluster 7: 103120 frames
Cluster 8: 124173 frames
Cluster 9: 17141 frames
'''